# Mapping federal crop insurance in the U.S.

A Jupyter notebook (Python 3) by Peter Donovan, managingwholes.com@gmail.com

Open data is not just a thing or a tool. It's a behavior, based on beliefs. This notebook is a way of sharing methods and assumptions, and if you use the same or similar tools (such as R instead of Python, for example) you can retread these steps. I hope this notebook may also serve as a guide for me as well as others who want to do similar things.

With crop insurance, as with any data set, looking at the data is a good way of learning about its particulars if not its intentions. Some knowledge of the context or domain of the data is usually required. 

For background on federal crop insurance, the following may be a start:

Dennis Shields' 2015 report from the Congressional Research Service: https://fas.org/sgp/crs/misc/R40532.pdf

Environmental Working Group's material on crop insurance, which includes interactive maps showing rate of return (payouts compared to premiums) on some crops by county from 2001 through 2014: http://www.ewg.org/research/crop-insurance-lottery. The average federal subsidy for crop insurance premiums is about 60%.

The Natural Resources Defense Council has a 2013 paper on crop insurance, https://www.nrdc.org/sites/default/files/soil-matters-IP.pdf. This paper suggests that crop insurance could be reformed to reward farming that is low risk with environmental rewards.

A starting hypothesis: federally subsidized crop insurance, while it sustains the economic viability of many farm businesses, might also tend to replace soil health and function as the foundation of a viable agriculture.

To investigate the hypothesis, we'll start by compiling data.

First, we get data. Download and unzip the data files from the USDA Risk Management Agency website: http://www.rma.usda.gov/data/cause.html The complete data for each year is under the "Summary of Business with Month of Loss" header.  So far I am using the 2001 through 2017 data. You can get the column headers from the same web page as a Word doc.

In [1]:
#some usual imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests, zipfile, io #for downloading files inside zipped archives
%matplotlib inline


From the website we see that years 1989 through 2017 are available as zip archives in Indemnities Only. (With a slower connection it may be better to download and extract the zip archives outside of this notebook.) Each zip file contains one text file such as colind09.txt, with a pipe character as separator.

Unzip a file and inspect it with a text editor. There are pipe characters separating the fields, and sometimes sequences of spaces before them or after them. There are no column headers.

What I'm planning is to map the total indemnities for each county (rows), for each year (columns), generate time series by county, and map columns as an animated choropleth of US counties. I'll test my code with one year's file and then try to use it as a function to add the other files' Indemnity Amount columns.

In [2]:
year = '2017'
#reading a csv inside a zipped file from a url

url = 'https://www.rma.usda.gov/data/col/indemnity/col_indem_'+ year + '.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
df = pd.read_csv(z.open(z.infolist()[0].filename), sep='|')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63500 entries, 0 to 63499
Data columns (total 14 columns):
2017                              63500 non-null int64
01                                63500 non-null int64
AL                                63500 non-null object
001                               63500 non-null int64
Autauga                           63500 non-null object
0011                              63500 non-null int64
WHEAT                             63500 non-null object
02                                63500 non-null int64
RP                                63500 non-null object
P2                                63500 non-null object
11                                63500 non-null object
Drought                           62914 non-null object
105.0000000000                    63500 non-null float64
3323.0000000000                   63500 non-null float64
dtypes: float64(2), int64(5), object(7)
memory usage: 6.8+ MB


In [21]:
df = pd.read_csv('../counties/RMA/colsom18.txt', sep='|')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115783 entries, 0 to 115782
Data columns (total 25 columns):
2018                                  115783 non-null int64
01                                    115783 non-null int64
AL                                    115783 non-null object
001                                   115783 non-null int64
Autauga                               115783 non-null object
0021                                  115783 non-null int64
COTTON                                115783 non-null object
02                                    115783 non-null int64
RP                                    115783 non-null object
A                                     115783 non-null object
H                                     115783 non-null object
31                                    115783 non-null object
Excess Moisture/Precipitation/Rain    114462 non-null object
12                                    115783 non-null int64
DEC                                   115

In [3]:
df.columns

Index(['2017', '01', 'AL', '001', 'Autauga                       ', '0011',
       'WHEAT                         ', '02', 'RP        ', 'P2', '11',
       'Drought', '105.0000000000', '3323.0000000000'],
      dtype='object')

In [5]:
#Column names are given in word files.

colind_cols_2001_2017=['Commodity Year','Location State Code','Location State Abbreviation ','Location County Code','Location County Name','Commodity Code','Commodity Name','Insurance Plan Code','Insurance Plan Abbreviation','Stage Code','Damage Cause Code','Damage Cause Description','Determined Acres','Indemnity Amount']

#the 1989 file has 14 fields, last one is all NaN so here's a guess:
colind_cols_1989_2000=['Commodity Year','Location State Code','Location State Abbreviation ','Location County Code','Location County Name','Commodity Code','Commodity Name','Insurance Plan Code','Insurance Plan Abbreviation','Stage Code','Damage Cause Code','Damage Cause Description','Indemnity Amount','NaN']

In [22]:
df.columns = ['Element Name','Crop Year Identifier','State Code','State Abbreviation','County Code','County Name','Crop Code','Crop Name','Insurance Plan Code','Insurance Plan Name Abbreviation','Coverage Category','Stage Code','Cause of Loss Code','Cause of Loss Description','Month of Loss','Month of Loss Name','Policies Earning Premium','Policies Indemnified','Net Planted Acres','Net Endorsed Acres','Liability','Total Premium','Subsidy,''Net Determined Acres','Indemnity Amount','Loss Ratio']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115783 entries, 0 to 115782
Data columns (total 25 columns):
Element Name                        115783 non-null int64
Crop Year Identifier                115783 non-null int64
State Code                          115783 non-null object
State Abbreviation                  115783 non-null int64
County Code                         115783 non-null object
County Name                         115783 non-null int64
Crop Code                           115783 non-null object
Crop Name                           115783 non-null int64
Insurance Plan Code                 115783 non-null object
Insurance Plan Name Abbreviation    115783 non-null object
Coverage Category                   115783 non-null object
Stage Code                          115783 non-null object
Cause of Loss Code                  114462 non-null object
Cause of Loss Description           115783 non-null int64
Month of Loss                       115783 non-null object
Month of L

In [24]:
df['Cause of Loss Description'].value_counts()

6     19717
7     17457
5     14135
8     12944
9     10969
10    10867
11     8774
4      7159
12     4766
3      3457
1      2920
2      2613
0         5
Name: Cause of Loss Description, dtype: int64

In [6]:
#Some of these causes have little relation to soil health. The main causes, however, do.
soilNOT = ['Decline in Price','Wildlife','Hurricane/Tropical Depression','Hail','Cyclone','Tornado','Earthquake','Volcanic Eruption']

#filter out damages clearly unrelated to soil health or function
df = df[~df['Damage Cause Description'].isin(soilNOT)]

AttributeError: 'float' object has no attribute 'strip'

### FIPS code
The state and county location codes are numeric (int64). FIPS (Federal Information Processing Standard) codes for counties are 5-digit strings. We'll pad with zeros using zfill function. This will come in handy when it comes to mapping, as we will want to merge or join our data with county boundaries using the FIPS code.

Alert data hounds will notice that the county code '999' is sometimes used in RMA data for 'All Other Counties' as a catch-all for (minor??) amounts in a state that are not bound to a county. How much are we dealing with?

In [109]:
#convert to strings, pad with zeros, 2 digits for state, 3 for county
df['st'] = df['Location State Code'].map(lambda x: str(x)).apply(lambda x: x.zfill(2))
df['co'] = df['Location County Code'].map(lambda x: str(x)).apply(lambda x: x.zfill(3))

#add id column (fips) and test
df['fips'] = df['st'] + df['co']
#df['fips][67]

In [110]:
#now we can narrow the columns to what we need. Note extra space after 'Location State Abbreviation'
df = df[['fips','Location State Abbreviation ','Location County Name', 'Indemnity Amount']]
df.columns = ['fips','st','name','indemnity']
#df.info()

In [122]:
g = df.groupby(['fips', 'st','name'],as_index=False).agg({'indemnity': np.sum})
g.indemnity = round(g.indemnity/1000000,2)
g.columns = ['fips','st','name','ind' + year]

In [125]:
g.describe()

,ind1989
count,2023.000000
mean,0.557271
std,1.178468
min,0.000000
25%,0.050000
50%,0.150000
75%,0.530000
max,19.260000


In [114]:
g.to_csv('ind'+ year + '.csv',index=False)

## Causes of loss
Let's look at the causes of loss. NOTE: These procedures could be duplicated to aggregate indemnities by 'Crop Name' as well.

In [9]:
df.groupby('Cause of Loss Description').agg({'Indemnity Amount':np.sum}).sort_values('Indemnity Amount',ascending=False)

,Indemnity Amount
Cause of Loss Description,
Excess Moisture/Precip/Rain,2483884562
Decline in Price,1848696139
Drought,1718087747
Hail,933202124
Cold Wet Weather,339836298
Area Plan Crops Only,316749929
Failure Irrig Supply,311330493
Freeze,273106811
Wind/Excess Wind,220228074


In [10]:
causes_2014 = df.groupby('Cause of Loss Description')['Indemnity Amount'].sum()
causes_2014.sort_values(ascending=False)

Cause of Loss Description
Excess Moisture/Precip/Rain          2483884562
Decline in Price                     1848696139
Drought                              1718087747
Hail                                  933202124
Cold Wet Weather                      339836298
Area Plan Crops Only                  316749929
Failure Irrig Supply                  311330493
Freeze                                273106811
Wind/Excess Wind                      220228074
Heat                                  203128308
Cold Winter                           109445405
Frost                                  78652859
Flood                                  75085010
Plant Disease                          59895203
Other (Snow-Lightning-Etc.)            45130916
Hot Wind                               40752799
Mycotoxin (Aflatoxin)                  28388250
Wildlife                               18875194
Hurricane/Tropical Depression           7931157
Insects                                 4835435
Failure Irrig 

In [11]:
#to generate a table of total indemnities by Cause of Loss, you can export a csv
causes_2014.to_csv('/Users/Peter/Documents/atlas/RMA/causes_2014.csv')

'Excess Moisture/Precip/Rain' and 'Drought' are by far the most common causes. Let's filter the dataframe by these two, so we can potentially see which counties had indemnities for both causes, and how much.

In [12]:
rain = df[df['Cause of Loss Description']=='Excess Moisture/Precip/Rain']
drought = df[df['Cause of Loss Description']=='Drought']
print(rain.shape, drought.shape)

(37767, 25) (22937, 25)


Now do a groupby on each dataframe by county, with sums of indemnity amounts.

In [13]:
g_rain = rain.groupby(['FIPS','County Name']).agg({'Indemnity Amount':np.sum})
g_drought = drought.groupby(['FIPS','County Name']).agg({'Indemnity Amount':np.sum})
together=pd.concat([g_rain,g_drought],axis=1)
together.columns = ['moisture','drought']
together.head()

,,moisture,drought
FIPS,County Name,,
01001,Autauga,"2,442.0","18,849.0"
01003,Baldwin,"816,341.0","116,282.0"
01005,Barbour,"20,835.0","588,264.0"
01007,Bibb,"22,572.0","43,286.0"
01011,Bullock,"60,122.0","77,797.0"


Let's add two columns, a total, and a ratio of moisture to drought.

In [14]:
together['total']=together.moisture + together.drought
together['ratio']=together.moisture / together.drought
together.head(20)

,,moisture,drought,total,ratio
FIPS,County Name,,,,
01001,Autauga,"2,442.0","18,849.0","21,291.0",0.1295559446124463
01003,Baldwin,"816,341.0","116,282.0","932,623.0",7.0203556870366866
01005,Barbour,"20,835.0","588,264.0","609,099.0",0.0354177716127453
01007,Bibb,"22,572.0","43,286.0","65,858.0",0.5214619045418842
01011,Bullock,"60,122.0","77,797.0","137,919.0",0.772806149337378
01013,Butler,"16,653.0","29,017.0","45,670.0",0.5739049522693593
01015,Calhoun,"36,100.0","15,935.0","52,035.0",2.265453404455601
01017,Chambers,nan,386.0,nan,nan
01019,Cherokee,"31,976.0","12,436.0","44,412.0",2.5712447732389836


In [15]:
mixed = together[(together.ratio < 4) & (together.ratio > .25)]
mixed.shape

(513, 4)

In [16]:
mixed.reset_index(level=0, inplace=True)
mixed.reset_index(level=0, inplace=True)
#run this twice

In [17]:
mixed = mixed.rename(columns={'total':'indemnity'})

In [18]:
mixed.indemnity = mixed.indemnity/1000000

In [19]:
mixed.to_csv('/Users/Peter/Documents/atlas/RMA/moisture_plus_drought_2014.tsv', sep='\t', index=False)

## Crop acreage
We may want to look at indemnities related to crop acreage in a county. The Farm Service Administration keeps track of cropland acreage. Data is available by year at https://www.fsa.usda.gov/news-room/efoia/electronic-reading-room/frequently-requested-information/crop-acreage-data/index. We'll start with 2016 crop year, aggregating acres by county.

In [17]:
df = pd.read_excel('/Users/Peter/Documents/atlas/RMA/2016_fsa_acres/2016_fsa_acres_010417.xlsx', sheetname='county_data')

In [21]:
df.columns

Index(['State Code', 'County Code', 'Crop Code', 'State', 'County',
       'State County Code', 'Crop', 'Crop Type', 'Intended Use',
       'Irrigation Practice', 'Planted Acres', 'Volunteer Acres',
       'Failed Acres', 'Prevented Acres', 'Not Planted Acres',
       'Planted and Failed Acres', 'acres'],
      dtype='object')

In [20]:
df['acres'] = df['Planted and Failed Acres']+df['Volunteer Acres']+df['Prevented Acres']

In [22]:
df = df[['State County Code','acres','Crop']]

In [23]:
corn_acres = df[df['Crop']=='CORN'].groupby('State County Code').agg({'acres':np.sum})

,acres
State County Code,
1001,"1,170.76"
1003,"7,606.080000000001"
1005,"2,350.97"
1007,27.2
1009,"1,587.3400000000001"
1011,571.47
1013,"1,463.1299999999999"
1015,840.98
1017,429.65999999999997


In [ ]:
corn_acres

In [26]:
df = df.rename(columns={'State County Code':'id'})

In [27]:
grouped = df.groupby('id').agg({'acres':np.sum})

In [58]:
grouped[grouped.acres > 4000000]

,acres
id,
4001,"4,847,170.78"
4005,"5,212,903.01"
32013,"6,103,906.630000002"
41025,"4,453,800.55"
56037,"4,478,592.130000001"


In [29]:
df2 = pd.read_csv('/Users/Peter/Documents/atlas/RMA/indemnity2016.tsv', sep='\t')
df2.set_index('id',inplace=True)
df2.shape

(2703, 2)

In [46]:
joined=pd.concat([df2,grouped],axis=1)

In [47]:
joined['ratio'] = joined.indemnity / joined['acres']*1000000

In [48]:
joined = joined.reset_index()

In [49]:
joined.id = joined.id.map(lambda x: str(x)).apply(lambda x: x.zfill(5))

In [50]:
joined = joined[['id','name','ratio']]

In [64]:
joined = joined.rename(columns={'index':'id'})

In [58]:
corn_by_acre = corn_by_acre.rename(columns={'index':'id'})

In [60]:

corn_by_acre[['id','name','indemnity']].to_csv('/Users/Peter/Documents/atlas/RMA/corn_by_acre.tsv',sep='\t')


In [61]:
corn_by_acre.indemnity.describe()

count                1,760.0
mean      17.857640729381185
std       27.673672436031378
min     0.029772117150757924
25%        2.699585695009991
50%        7.912187470332242
75%        21.73488160711495
max        400.6666666666667
Name: indemnity, dtype: float64

In [67]:
joined.indemnity.describe()

count               2,666.0
mean      18.36627861759955
std       166.9844856142665
min     0.00420908323753347
25%       1.553177194237957
50%       4.205278759792259
75%      12.999815226664863
max      6,980.731364275668
Name: indemnity, dtype: float64

In [66]:
joined.to_csv('/Users/Peter/Documents/atlas/RMA/indemnity_per_acre_2016.tsv', sep='\t')

In [14]:
cols_w_spaces = ['County Name','Crop Name','Insurance Plan Name Abbreviation','Cause of Loss Description']

for item in cols_w_spaces:
    df16[item] = df16[item].map(lambda x: x.strip())


In [15]:
df16g = df16.groupby('Crop Name').agg({'Indemnity Amount':np.sum})

In [20]:
df16g.sort_values('Indemnity Amount', ascending=False)

,Indemnity Amount
Crop Name,
CORN,"935,682,263.0000004"
WHEAT,"493,662,051.9999991"
SOYBEANS,"373,826,331.9999997"
COTTON,"302,464,387.00000024"
All Other Crops,"177,054,364.99999994"
"PASTURE,RANGELAND,FORAGE","143,857,349.0"
FLUE CURED TOBACCO,"126,479,742.00000006"
RICE,"104,175,766.0"
BURLEY TOBACCO,"84,075,077.0"


In [16]:
df16g[df16g['Crop Name']=='PASTURE, RANGELAND, FORAGE']

KeyError: 'Crop Name'